# Uncertainty Forest (UF) Demo
This demo provides the basic use cases of the `uncertainty_forest` module.

In [1]:
import numpy as np
from uncertainty_forest.uncertainty_forest import UncertaintyForest

## Hyperparameter Specification
Random forest hyperparameters such as minimum leaf size and maximum depth can be specified by the UF constructor.

In [2]:
# Notation from the current paper. 
max_depth=30,           # D
min_samples_leaf=1,     # k
max_features = None,    # m
n_trees = 200,          # B
max_samples = .5,       # s // 2
bootstrap = False       # Whether to subsample with replacement.

uf = UncertaintyForest(
    max_depth = max_depth,          
    min_samples_leaf = min_samples_leaf,    
    max_features = max_features,   
    n_trees = n_trees,   
    max_samples = max_samples,   
    bootstrap = bootstrap,
)

In [3]:
# Or, you can just use the defaults.
uf = UncertaintyForest()

## Estimate the Conditional Probability of `Y` given `X = x`
Use `X_train` and `y_train` to estimate the posterior, and evaluate this posterior at `X_eval`.

In [4]:
n_class = 25
d = 10
classes = [-1, 0, 1]

X_train = np.concatenate([np.random.multivariate_normal(k*np.ones(d), np.eye(d), n_class) for k in classes])
y_train = np.concatenate([k*np.ones(n_class) for k in classes])
print(X_train.shape)
print(y_train.shape)

X_eval = np.array([c*np.ones(d) for c in range(-2, 3)])
print(X_eval)

(75, 10)
(75,)
[[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]]


In [5]:
cond_probability = uf.estimate_cond_probability(X_train, y_train, X_eval)
print("P(Y|X = x) for x in X_eval:")
print(cond_probability)

P(Y|X = x) for x in X_eval:
[[0.75640096 0.16467976 0.07891928]
 [0.75140511 0.17232061 0.07627428]
 [0.28677663 0.49333013 0.21989323]
 [0.11789585 0.27709984 0.60500432]
 [0.09812507 0.19592294 0.70595198]]


## Estimate the Conditional Entropy
If no data is supplied, `H(Y | X)` is computed using the stored conditional probability (of the evaluation set `X_eval`).

In [6]:
cond_entropy = uf.estimate_cond_entropy()
print("0 <= H(Y|X) = %f <= log2(3) = %f" % (cond_entropy, np.log2(3.0)))

0 <= H(Y|X) = 1.202395 <= log2(3) = 1.584963


New data can be supplied directly to the `estimate_cond_entropy` function. You must provide **both** training data and evaluation data.

In [7]:
n_class = 10
d = 2

classes = range(1, 5) # Classes can be arbitrary, will be preprocessed into {0, 1, ..., K}

X_train = np.concatenate([np.random.multivariate_normal(k*np.ones(d), 0.5*k*np.eye(d), n_class) for k in classes])
y_train = np.concatenate([k*np.ones(n_class) for k in classes])
X_eval = np.array([c*np.ones(d) for c in range(-2, 3)])

# Notation from the current paper. 
n_trees = 1

uf = UncertaintyForest(  
    n_trees = n_trees
)

cond_entropy = uf.estimate_cond_entropy(X_train, y_train, X_eval)
print("H(Y|X) = %f" % cond_entropy)

H(Y|X) = 1.699575


# Estimate the mutual information
Mutual information can be estimated with the exact same schema as conditional entropy, with `estimate_mutual_info`.

In [8]:
mutual_info = uf.estimate_mutual_info()
print("I(X, Y) = %f" % mutual_info)

I(X, Y) = 0.300425
